# LRIF(Loop Random Intersection Forest) V4.2  

In [ ]:
import time
# 参数
VERSION = '4.2'
X_INPUT_NAME = 'Features_94_343.csv'
Y_INPUT_NAME = 'Values_94.csv'
TITLE_INPUT_NAME = 'Title_343.csv'
RECEIVE_THRESHOLD = 12
TREE_NUM = 20000
BRANCH_NUM = 3
MAX_LEVEL = 10
RATIO_1 = 0.90
RATIO_2 = 0.20
RATIO_3 = 0.20
RATIO_4 = 0.80
RATIO_5 = 0.00
INDEX_FIT = True
WEIGHT_WAY = 4
SUPTITLE = 'Importance of Features from LRIF V'+VERSION+'\nDate: '+time.strftime("%Y%m%d_%H%M%S", time.localtime())
DIR_NAME = 'LRIF_'+time.strftime("%Y%m%d_%H%M%S", time.localtime())+'--Version '+VERSION+'--'
PLOT_NAME = 'LRIF_V'+VERSION+'_'+time.strftime("%Y%m%d_%H%M%S", time.localtime())+'.png'
LOG_NAME = 'Log_LRIF_V'+VERSION+'_'+time.strftime("%Y%m%d_%H%M%S", time.localtime())+'.txt'
DISPLAY_NUM = 30
REMAIN_NUM = 1200

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X = np.loadtxt(X_INPUT_NAME, delimiter=',').astype(int)
y = np.loadtxt(Y_INPUT_NAME).astype(int)
title = np.loadtxt(TITLE_INPUT_NAME, dtype=str)
print(X.shape, y.shape, title.shape)

In [ ]:
def intersection(x1, x2, index, flag):
    global RATIO_1, RATIO_2, RATIO_3, RATIO_4, RATIO_5
    new_index = np.zeros((x1.shape[0], ))
    for i in range(x1.shape[0]):
        if index[i, ] == 1:
            if flag:
                if x1[i, ] == x2[i, ]:
                    if np.random.rand() <= RATIO_1:
                        new_index[i, ] = 1
                else:
                    if x1[i, ] == 1:
                        if np.random.rand() <= RATIO_2:
                            new_index[i, ] = 1
                    else:
                        if np.random.rand() <= RATIO_3:
                            new_index[i, ] = 1
            else:
                if x1[i, ] == x2[i, ]:
                    if np.random.rand() >= RATIO_4:
                        new_index[i, ] = 1
                else:
                    new_index[i, ] = 1
        else:
            if flag:
                if x2[i, ] == x1[i, ]:
                    if np.random.rand() <= RATIO_5:
                        new_index[i, ] = 1
            else:
                if x2[i, ] != x1[i, ]:
                    if np.random.rand() <= RATIO_5:
                        new_index[i, ] = 1
    return new_index

In [ ]:
Out = []
X_label_1 = X[np.argwhere(y==1).flatten().tolist(), :]
X_label_0 = X[np.argwhere(y==0).flatten().tolist(), :]
leaf_sum = []
level_sum = []
for _ in range(TREE_NUM):
    leaf_count = 0
    none_leaf_count = 0
    node_count = 1
    X_1 = X_label_1[np.random.randint(0, X_label_1.shape[0]), :]
    index_list = []
    for level in range(MAX_LEVEL):
        next_node_count = 0
        if node_count == 0:
            print('One tree done! Total leaf:', leaf_count, ',Total none leaf:', none_leaf_count, ',Total level:', level+1, 
                  '\nTotal:', _+1, 'tree(s) has(have) finished their mission.\n')
            break
        next_index_list = []
        if level == 0:
            if INDEX_FIT:
                index_list.append(X_1)
            else:
                index_list.append(np.ones((X_1.shape[0], )))
        for node in range(node_count):
            k = np.random.randint(0, X.shape[0])
            X_2 = X[k, :]
            f = True if y[k, ] == 1 else False
            node_out = intersection(X_1, X_2, index_list[node], f)
            if (sum(node_out>0) <= RECEIVE_THRESHOLD):
                if sum(node_out) > 0:
                    Out.append(node_out)
                leaf_count += 1
            else:
                next_node_count += BRANCH_NUM
                none_leaf_count += 1
                for i in range(BRANCH_NUM):
                    next_index_list.append(node_out)
        node_count = next_node_count
        index_list = next_index_list
    print('One tree done! Total leaf:', leaf_count, ',Total none leaf:', none_leaf_count, ',Total level:', level+1, 
          '\nTotal:', _+1, 'tree(s) has(have) finished their mission.\n')
    leaf_sum.append(leaf_count)
    level_sum.append(level+1)

In [ ]:
print('Total number of effective leaf output:', len(Out))
count = np.zeros((X.shape[1], ))
for out in Out:
    for i in range(len(out)):
        if out[i]>0:
            count[i, ] += 1.0

In [ ]:
count = count.astype(float)
for i in range(count.shape[0]):
    if WEIGHT_WAY==0:
        temp = 1
    elif WEIGHT_WAY==1:
        temp = sum(X[:, i])
    elif WEIGHT_WAY==2:
        if sum(X[:, i])>1:
            temp = 0.5*sum(X[:, i])*(sum(X[:, i])-1)
        else:
            temp = 1
    elif WEIGHT_WAY==3:
        if sum(X_label_1[:, i])>0:
            temp = sum(X[:, i])*sum(X_label_1[:, i])
        else:
            temp = sum(X[:, i])
    elif WEIGHT_WAY == 4:
        temp_list = []
        temp_list.append((sum(X_label_1[:, i])**2+(X_label_1.shape[0]-sum(X_label_1[:, i]))**2)*RATIO_1)
        temp_list.append((X_label_1.shape[0]-sum(X_label_1[:, i]))*sum(X_label_1[:, i])*RATIO_2)
        temp_list.append(sum(X_label_1[:, i])*(X_label_1.shape[0]-sum(X_label_1[:, i]))*RATIO_3)
        temp_list.append((sum(X_label_1[:, i])**2+(X_label_1.shape[0]-sum(X_label_1[:, i]))**2)*RATIO_5)
        temp_list.append((sum(X_label_0[:, i])*sum(X_label_1[:, i])+(X_label_0.shape[0]-sum(X_label_0[:, i]))*(X_label_1.shape[0]-sum(X_label_1[:, i])))*RATIO_4)
        temp_list.append(sum(X_label_1[:, i])*(X_label_0.shape[0]-sum(X_label_0[:, i]))+sum(X_label_0[:, i])*(X_label_1.shape[0]-sum(X_label_1[:, i])))
        temp_list.append((sum(X_label_0[:, i])*(X_label_1.shape[0]-sum(X_label_1[:, i]))+(X_label_0.shape[0]-sum(X_label_0[:, i]))*sum(X_label_1[:, i]))*RATIO_5)
        temp = sum(temp_list)
    count[i, ] = count[i, ]*X.shape[0]*X_label_1.shape[0]/temp

In [ ]:
p = np.argsort(-count)
score = []
score_count = []
sum_temp = sum(count)
for i in range(len(p)):
    score_ratio = 100*count[p[i], ]/sum_temp
    score.append(score_ratio)
    score_count.append(sum(score))

In [ ]:
import os
from pathlib import Path
DIR = DIR_NAME
os.mkdir(DIR)
PLOT_NAME = Path('.', DIR, PLOT_NAME)
LOG_NAME = Path('.', DIR, LOG_NAME)

In [ ]:
plt.figure(figsize=(22,14), dpi=200)
plt.subplot(211)
plt.bar(np.linspace(1, len(p), len(p)), count[:, ])
plt.title('Appearance score of features', fontsize=17)
plt.ylabel('Appearance score of features')
plt.xlabel('Index')
plt.subplot(212)
plt.tick_params(axis='x', labelsize=5)
plt.bar(title[p[:DISPLAY_NUM, ], ].flatten().tolist(), count[p[:DISPLAY_NUM, ], ].flatten().tolist())
plt.title(str(DISPLAY_NUM)+' Max Coef', fontsize=17)
plt.ylabel('Appearance score of features')
plt.xlabel('Feature Name')
plt.suptitle(SUPTITLE, fontsize=20)
plt.savefig(PLOT_NAME, bbox_inches='tight')

In [ ]:
f1 = open(LOG_NAME, 'w+')
f1.write('Output of Loop Random Intersection Forest:\n\n')
f1.write('Data: '+X_INPUT_NAME+'  '+Y_INPUT_NAME+'  '+TITLE_INPUT_NAME+'\n')
f1.write('Data Shape: '+str(X.shape)+', '+str(y.shape)+'\n\n')
f1.write('Threshold: '+str(RECEIVE_THRESHOLD)+'\n')
f1.write('Tree Num: '+str(TREE_NUM)+'\n')
f1.write('Node Num(new branch, per node): '+str(BRANCH_NUM)+'\n')
f1.write('Max level: '+str(MAX_LEVEL)+'\n')
f1.write('Ratio 1: '+str(RATIO_1)+'\n')
f1.write('Ratio 2: '+str(RATIO_2)+'\n')
f1.write('Ratio 3: '+str(RATIO_3)+'\n')
f1.write('Ratio 4: '+str(RATIO_4)+'\n')
f1.write('Ratio 5: '+str(RATIO_5)+'\n')
if INDEX_FIT:
    f1.write('First index input: Same as X_1\n\n')
else:
    f1.write('First index input: All 1\n\n')
f1.write('Total leaf node: '+str(sum(leaf_sum))+'\n')
f1.write('Average leaf node: '+str(sum(leaf_sum)/TREE_NUM)+' per tree\n')
f1.write('Total level: '+str(sum(level_sum))+'\n')
f1.write('Average level: '+str(sum(level_sum)/TREE_NUM)+' per tree\n\n')
f1.write('Total number of effective leaf output: '+str(len(Out))+'\n\n\n')
f1.write(str(DISPLAY_NUM)+' Maximum Features:\n')
for i in range(DISPLAY_NUM):
    f1.write('Name: '+str(title[p[i, ], ])+'   Time-Count-Score: '+str(count[p[i, ], ])+
             '   Importance: '+str(score[i])+'%   Sum of Importance: '+str(score_count[i])+'%\n')

In [ ]:
print('Total number of effective leaf output:', len(Out))
set_list = np.unique(Out, axis=0)
print(set_list.shape)
# for out in Out:
#     flag = True
#     for s in set_list:
#         if out.flatten().tolist() is s:
#             flag = False
#             break
#     if flag:
#         set_list.append(out)
print('Total number of unique output:', len(set_list))

In [ ]:
pi = sum(y)/X.shape[0]
print(pi)
def cal_i_score(set_in, x_data, y_data):
    global pi
    index_temp = np.argwhere(set_in==1).flatten().tolist()
    list_temp = []
    i_s = 0.0
    for i in range(x_data.shape[0]):
        flag = True
        for f in list_temp:
            if x_data[i, index_temp].flatten().tolist() == f.tolist():
                flag = False
                break
        if flag:
            list_temp.append(x_data[i, index_temp])
    for l in list_temp:
        l_count = 0.0
        l_1_count = 0.0
        for i in range(x_data.shape[0]):
            if x_data[i, index_temp].flatten().tolist() == l.tolist():
                l_count += 1.0
                if y_data[i] == 1:
                    l_1_count +=1.0
        i_s += (l_1_count-pi*l_count)**2
    return i_s

In [ ]:
i_score = []
for s in set_list:
    i_score_temp = cal_i_score(s, X, y)
    i_score.append(i_score_temp)
y_std = np.var(y)
d_temp = y_std*X.shape[0]

In [ ]:
p = np.argsort(-np.array(i_score).reshape(len(i_score), )).flatten().tolist()
new_list = []
new_score_list = []
f1.write('\n\n\nThe most effective '+str(REMAIN_NUM)+' featrue-sets to be put into the next greedy dropping step:\n\n')
for i in range(REMAIN_NUM):
    new_list.append(set_list[p[i]])
    new_score_list.append(i_score[p[i]])
    f1.write('Feature list: '+str(title[np.argwhere(new_list[i]==1), ].flatten().tolist())+'\n   I-Score: '+str(new_score_list[i]/d_temp)+'\n\n')

In [ ]:
out_list = []
out_score = []
for i in range(len(new_list)):
    l = new_list[i]
    old_score = new_score_list[i]
    continue_flag = True
    while continue_flag:
        next_score = []
        next_delete = []
        filled_pos = np.argwhere(l==1).flatten().tolist()
        for p in filled_pos:
            next_delete.append(p)
            l[p] = 0
            next_score.append(cal_i_score(l, X, y))
            l[p] = 1
        if max(next_score) >= old_score:
            l[next_delete[next_score.index(max(next_score))]] = 0
            old_score = max(next_score)
        else:
            continue_flag = False
            out_list.append(title[np.argwhere(l==1), ].flatten().tolist())
            out_score.append(old_score)
p = np.argsort(-np.array(out_score).reshape(len(out_score), ))
f1.write('\n\n\nThe most effective featrue-sets after greedy dropping:\n\n')
old = []
for i in p:
    if str(out_list[i]) not in old:
        f1.write('Feature list: '+str(out_list[i])+'\n   I-Score: '+str(out_score[i]/d_temp)+'\n\n')
        old.append(str(out_list[i]))
    else:
        continue
f1.close()